# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096818


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:50,  3.81s/it]

Rendering models:  10%|▉         | 3/31 [00:08<01:21,  2.90s/it]

Rendering models:  13%|█▎        | 4/31 [00:09<01:04,  2.40s/it]

Rendering models:  16%|█▌        | 5/31 [00:10<00:49,  1.89s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:26<00:01,  1.62it/s]

Rendering models:  94%|█████████▎| 29/31 [00:27<00:01,  1.36it/s]

Rendering models:  97%|█████████▋| 30/31 [00:27<00:00,  1.55it/s]

Rendering models: 100%|██████████| 31/31 [00:27<00:00,  1.69it/s]

Jnursssmith5263                        0.000993
not-logged-in-f45cae824435ec815f31     0.057589
Alexander00                            0.000220
universehead-1                         0.000218
Jonas_Cross                            0.000310
not-logged-in-7edd9b2ab9752961abc1     0.078981
jnarayanbvg                            0.023153
c196948                                0.015759
Quarkgc                                0.000265
not-logged-in-e7932b1588574fe4dae2     0.008520
mr_willock                             0.000293
not-logged-in-9a45911eb71184fe7c7f     0.000263
Giantjack14                           59.597102
acapirala                              0.000318
vctjones                               0.057269
Lavadude                               0.041087
not-logged-in-8ed37f989908c6fada03     0.002191
lgonzalez62                            0.003004
tdanyluk                               0.000161
tdanyluk                               0.000196
IdeasInSpace                           0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())